In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time

In [2]:
df = pd.read_csv('ToN-IoT_balanced_smote.csv')

In [3]:
df

,src_port,dst_port,duration,src_bytes,dst_bytes,missed_bytes,src_ip_bytes,dst_ip_bytes,dns_rcode,label,...,weird_name_dnp3_corrupt_header_checksum,weird_name_inappropriate_FIN,weird_name_possible_split_routing,weird_addl_43,weird_addl_46,weird_addl_48,weird_addl_None,weird_notice_F,weird_notice_None,type
0,0.101952,3.799303,175.726962,0.152327,3.438007,0.0,86.672008,0.001470,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
1,2.558822,1.118792,0.000000,0.000000,0.000000,0.0,0.000366,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,False,True,5.0
2,0.101790,3.713672,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
3,1.856906,0.753177,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,False,False,False,False,False,False,True,False,True,5.0
4,2.523521,0.003801,0.000001,0.000000,0.000024,0.0,0.000000,0.002205,0.0,0.0,...,False,False,False,False,False,False,True,True,False,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,3.279039,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999996,1.823237,0.003801,0.000024,0.000007,0.000009,0.0,0.000791,0.001046,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999997,2.563471,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0
2999998,1.853248,0.005737,0.000000,0.000000,0.000000,0.0,0.000302,0.000000,0.0,1.0,...,False,False,False,False,False,False,True,False,True,9.0


In [6]:
X = df.drop(['type','label'], axis=1)
y = df['type']
# one hot encoding
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential()
model.add(Conv1D(64, 3,activation="relu",input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(64, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128, 3, activation="relu"))
model.add(Conv1D(128, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

start = time.time()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[mc])
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
75000/75000 [==============================] - ETA: 0s - loss: 0.2395 - accuracy: 0.9326
Epoch 1: val_accuracy improved from -inf to 0.94677, saving model to cnn_best_model.h5
75000/75000 [==============================] - 1416s 19ms/step - loss: 0.2395 - accuracy: 0.9326 - val_loss: 0.1776 - val_accuracy: 0.9468
Epoch 2/10
    5/75000 [..............................] - ETA: 35:46 - loss: 0.1563 - accuracy: 0.9500

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


74999/75000 [============================>.] - ETA: 0s - loss: 0.1340 - accuracy: 0.9659
Epoch 2: val_accuracy improved from 0.94677 to 0.97168, saving model to cnn_best_model.h5
75000/75000 [==============================] - 1461s 19ms/step - loss: 0.1340 - accuracy: 0.9659 - val_loss: 0.1017 - val_accuracy: 0.9717
Epoch 3/10
74997/75000 [============================>.] - ETA: 0s - loss: 0.1190 - accuracy: 0.9692
Epoch 3: val_accuracy did not improve from 0.97168
75000/75000 [==============================] - 1005s 13ms/step - loss: 0.1190 - accuracy: 0.9692 - val_loss: 0.0987 - val_accuracy: 0.9715
Epoch 4/10
74998/75000 [============================>.] - ETA: 0s - loss: 0.1118 - accuracy: 0.9712
Epoch 4: val_accuracy improved from 0.97168 to 0.97617, saving model to cnn_best_model.h5
75000/75000 [==============================] - 764s 10ms/step - loss: 0.1118 - accuracy: 0.9712 - val_loss: 0.0881 - val_accuracy: 0.9762
Epoch 5/10
75000/75000 [==============================] - ETA: 0

In [7]:
model = load_model('cnn_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

18750/18750 [==============================] - 59s 3ms/step
Time taken to predict:  62.6370370388031
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60144
           1       0.95      0.96      0.96     60079
           2       1.00      0.97      0.98     60047
           3       0.98      0.94      0.96     59974
           4       0.95      0.99      0.97     60175
           5       0.98      0.97      0.97     59762
           6       0.99      0.98      0.99     59631
           7       0.99      1.00      0.99     59909
           8       0.99      0.98      0.98     59992
           9       0.95      0.99      0.97     60287

    accuracy                           0.98    600000
   macro avg       0.98      0.98      0.98    600000
weighted avg       0.98      0.98      0.98    600000

0.97735


In [8]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
for i in range(cm.shape[0]):
    print(i, cm[i][i])

Accuracy of each class:
0 0.9994180633147114
1 0.9610512824780706
2 0.9691908005395773
3 0.9401907493247074
4 0.991657665143332
5 0.9701817208259429
6 0.9832637386594221
7 0.997813350247876
8 0.9755300706760901
9 0.9851211704015791
